In [121]:
import numpy as np
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv(r'C:\Users\User\Desktop\IM PR & ML\ML Couse\Code_pro\Project\laptop_price.csv',encoding='latin1')
data['Ram'] = data['Ram'].str.replace('GB','').astype('int32')
data['Weight'] = data['Weight'].str.replace('kg','').astype('float32')

def add_company(inpt):
    if inpt == 'Samsung' or inpt == 'Razer' or inpt == 'Mediacom' or inpt == 'Microsoft'or inpt == 'Xiaomi'or inpt == 'Vero'or inpt == 'Chuwi'or inpt == 'Google'or inpt == 'Fujitsu'or inpt == 'LG'or inpt == 'Huawei':
        return 'Other'
    else:
        return inpt
data['Company'] = data['Company'].apply(add_company)
data['Touchscreen'] = data['ScreenResolution'].apply(lambda x:1 if 'Touchscreen' in x else 0)
data['Ips'] = data['ScreenResolution'].apply(lambda x:1 if 'IPS' in x else 0)
data['cpu_name'] = data['Cpu'].apply(lambda x:" ".join(x.split()[0:3]))

def set_processor(name):
    if name == 'Intel Core i7' or name == 'Intel Core i5' or name == 'Intel Core i3':
        return name
    else:
        if name.split()[0] == 'AMD':
            return 'AMD'
        else:
            return 'Other'
data['cpu_name']=data['cpu_name'].apply(set_processor)
data['gpu_name'] = data['Gpu'].apply(lambda x:" ".join(x.split()[0:1]))
data = data[data['gpu_name'] != 'ARM']

def set_os(inpt):
    if inpt == 'Windows 10' or inpt == 'Windows 7' or inpt == 'Windows 10 S':
        return 'Windows'
    elif inpt == 'macOS' or inpt == 'Mac OS X':
        return 'Mac'
    elif inpt == 'Linux':
        return inpt
    else:
        return 'Other'

data['OpSys'] = data['OpSys'].apply(set_os)
data = data.drop(columns=['laptop_ID', 'Inches', 'Product', 'ScreenResolution', 'Cpu', 'Gpu'])

data = pd.get_dummies(data)

x=data.drop('Price_euros',axis=1) #remove the column
y=data['Price_euros']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

def model_acc(model):
    model.fit(X_train, y_train)
    acc = model.score(X_test, y_test)
    print(str(model)+ ' --> ' +str(acc))

    from sklearn.linear_model import LinearRegression
lr = LinearRegression()
model_acc(lr)

from sklearn.linear_model import Lasso
lasso = Lasso()
model_acc(lasso)

from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()
model_acc(dt)

from sklearn.ensemble import RandomForestRegressor  #High accuracy and I get the this model in this problem
rf = RandomForestRegressor()
model_acc(rf)

LinearRegression() --> 0.7160912318294048
Lasso() --> 0.7156303223705249
DecisionTreeRegressor() --> 0.7074509799045942
RandomForestRegressor() --> 0.7926400075515814


In [122]:
from sklearn.model_selection import GridSearchCV

parameters = {'n_estimators':[10, 50, 100],
              'criterion':['squared_error','absolute_error','poisson']}

grid_obj = GridSearchCV(estimator=rf, param_grid=parameters)

grid_fit = grid_obj.fit(X_train, y_train)

best_model = grid_fit.best_estimator_

best_model.score(X_test, y_test)

0.8032589494919462

In [123]:
import pickle
with open('predictor.pickle', 'wb') as file:
    pickle.dump(best_model, file)